# Model Inheritance

As we discussed in the lecture, model inheritance can be used to split your models into separate pieces - possibly for re-use. 

One primary purpose for inheritance is to have a standardized customized base model.

For example, suppose we want all our models to:
- ignore extra fields
- use autogenerated aliases
- allow populateion by both name and alias

Although we could certainly define these model configs on each and every model, this is not ideal.

Instead, we can create a custom base model (with no fields, just a config):

In [1]:
from pydantic import BaseModel, ConfigDict
from pydantic.alias_generators import to_camel

In [2]:
class CustomBaseModel(BaseModel):
    model_config = ConfigDict(
        extra="ignore",
        alias_generator=to_camel, 
        populate_by_name=True
    )

Now let's go back to our last composition example, and use our custom base model instead:

In [3]:
from typing import Annotated

from pydantic import AfterValidator, EmailStr, Field, PastDate


SortedStringList = Annotated[list[str], AfterValidator(lambda value: sorted(value, key=str.casefold))]

class ContactInfo(CustomBaseModel):
    email: EmailStr | None = None

class PlaceInfo(CustomBaseModel):
    city: str
    country: str
    
class PlaceDateInfo(CustomBaseModel):
    date_: PastDate = Field(alias="date")
    place: PlaceInfo
    
class PersonalInfo(CustomBaseModel):
    nationality: str
    born: PlaceDateInfo

class Person(CustomBaseModel):
    first_name: str
    last_name: str
    contact_info: ContactInfo
    personal_info: PersonalInfo
    notable_students: SortedStringList = []

In [4]:
json_data = """
{
    "firstName": "David",
    "lastName": "Hilbert",
    "contactInfo": {
        "email": "d.hilbert@spectral-theory.com",
        "homePhone": {
            "countryCode": 49,
            "areaCode": 551,
            "localPhoneNumber": 123456789
        }
    },
    "personalInfo": {
        "nationality": "German",
        "born": {
            "date": "1862-01-23",
            "place": {
                "city": "Konigsberg",
                "country": "Prussia"
            }
        },
        "died": {
            "date": "1943-02-14",
            "place": {
                "city": "Gottingen",
                "country": "Germany"
            }
        }
    },
    "awards": ["Lobachevsky Prize", "Bolyai Prize", "ForMemRS"],
    "notableStudents": ["von Neumann", "Weyl", "Courant", "Zermelo"]
}
"""

In [5]:
p = Person.model_validate_json(json_data)

In [6]:
print(p.model_dump_json(by_alias=True, indent=2))

{
  "firstName": "David",
  "lastName": "Hilbert",
  "contactInfo": {
    "email": "d.hilbert@spectral-theory.com"
  },
  "personalInfo": {
    "nationality": "German",
    "born": {
      "date": "1862-01-23",
      "place": {
        "city": "Konigsberg",
        "country": "Prussia"
      }
    }
  },
  "notableStudents": [
    "Courant",
    "von Neumann",
    "Weyl",
    "Zermelo"
  ]
}


Another use for inheritance might be because you want all your models to contain certain fields.

For example, you might be creating a REST API, and you want every response from your API to include some basic information about the request: maybe a unique ID, the date and time the request was made, and how long it took to execute.

We can use inheritance for that. A word of caution though, do not get tempted by multiple inheritance, where you have models that inherit from multiple models - unless you're an expert in multiple inheritance, and know that the way you intend to use it is supported by Pydantic, you will run into issues. (as an example, I ran into several major issues because I was using multiple inheritance in Pydantic V1 to tie together a Pydantic model with a corresponding Dynamo table and all the CRUD functionality for it - my implementation completely broke once I moved to V2, and that was a rather painful refactor, and I ended up using composition instead.)

However, we can leverage single inheritance rather well for a scenario such as the one I described above.

Here's one approach we might take.

We'll start with a custom base, with no fields - we might want the configuration to be the same for all our models, but not all our models in our app will necessarily require the standard request info fields we discussed.

So, I will keep those things separate.

We still have our custom base model:

In [7]:
class CustomBaseModel(BaseModel):
    model_config = ConfigDict(
        extra="ignore",
        alias_generator=to_camel, 
        populate_by_name=True
    )

Next, we create some models for the request info. I am going to grab some of the re-usable annotated types we created for date handling, as well as the JSON serialization we implemented for datetime objects. This is nothing new, we covered this already in previous sections.

In [8]:
from datetime import datetime
from typing import Any

import pytz
from dateutil.parser import parse
from pydantic import AfterValidator, BeforeValidator, FieldSerializationInfo, field_serializer, PlainSerializer


def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt
    
def parse_datetime(value: Any):
    if isinstance(value, str):
        try:
            return parse(value)
        except Exception as ex:
            raise ValueError(str(ex))
    return value

def dt_serializer(dt, info: FieldSerializationInfo) -> datetime | str:
    if info.mode_is_json():
        return dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    return dt
    
DateTimeUTC = Annotated[
    datetime, 
    BeforeValidator(parse_datetime), 
    AfterValidator(make_utc), 
    PlainSerializer(dt_serializer, when_used="unless-none")
]

Now we'll create our base model for responses:

In [9]:
from uuid import uuid4

class RequestInfo(CustomBaseModel):
    query_id: uuid4 = Field(default_factory=uuid4)
    execution_dt: DateTimeUTC = Field(default_factory=lambda: datetime.now(pytz.utc))
    elapsed_time_secs: float

class ResponseBaseModel(CustomBaseModel):
    request_info: RequestInfo    

As you can see, `RequestInfo` and `ResponseBaseModel` both inherited from `CustomBaseModel`, so they have our default model configuration.

And now, we can use this `ResponseBaseModel` as the base for all our response models in our API.

In [10]:
class Users(ResponseBaseModel):
    users: list[str] = []

In [11]:
users = Users(request_info=RequestInfo(elapsed_time_secs=3.14), users=["Athos", "Porthos", "Aramis"])

In [12]:
print(users.model_dump_json(by_alias=True, indent=2))

{
  "requestInfo": {
    "queryId": "c76169d9-7be0-481f-b679-6a9f2f251c77",
    "executionDt": "2023-12-02T18:27:09Z",
    "elapsedTimeSecs": 3.14
  },
  "users": [
    "Athos",
    "Porthos",
    "Aramis"
  ]
}
